<a href="https://colab.research.google.com/github/Npkh/BlueStars-Tools/blob/main/Auto_L%C3%A0m_D%E1%BB%AF_Li%E1%BB%87u_B%C3%A1o_C%C3%A1o_Dashboard_BlueStars.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Cách Thực Hiện Auto Làm Dữ Liệu Dashboard**
*   B1: Tải File checklist sản phẩm về (file này là checklist sản phẩm, có thể cần cập nhật trong tương lai hoặc trong quá trình lọc thiếu sản phẩm. File này chỉ tải về 1 lần duy nhất là lần đầu sử dụng code. Sau đó lưu và cập nhật tên sản phẩm trực tiếp vào file này trong tương lai)

[***File checklist sản phẩm***](https://docs.google.com/spreadsheets/d/1OeXnWLvYYZM34Wyf7EzW3bnCnsrw1T7f/edit?usp=sharing&ouid=105336776683915595895&rtpof=true&sd=true) - Tải file .xlsx

*   B2: Copy dữ liệu của data (SB, SD, SP) sau khi tải về vào sheet "data", sheet này cần được clear data mỗi lần thực hiện xong báo cáo

*   B3: Lưu và upload file này lên trên đoạn code này, bên dưới có option "Upload file"

*   B4: Kết quả tải về cần format lại định dạng một số cột như:
  - Spend, Sale sang dạng currency

*   B5: Kiểm tra xem cột ***Product Number*** và ***SKU*** có giá trị "*None*" hay không? Nếu có cần cập nhật lại checklist sản phẩm lưu lại rồi có thể chạy lại đoạn code này hoặc điền thủ công nếu có ít giá trị trống

***Lưu Ý***:
*   Đoạn code sẽ tự động chạy và tải về kết quả, nếu không có kết quả tải về thì đoạn code đã bị lỗi, cần check lại các bước đã làm đúng hay chưa. Thời gian chạy code khoảng 7-30 phút (Tùy vào độ nặng của file)

*   Nếu chẳng may xóa một đoạn code nào đó, reload lại trang này để khôi phục code

*   Phải đăng nhập google drive mới có thể chạy Run all

*   Các cột có giá trị là **currency** đã được tự động *0.76 khi nhập market là Canada

*   Làm từng data một, không copy hết vào chung sheet data. Làm xong data nào thì clear data ở sheet "data" và paste data mới vào




In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime
from google.colab import files
import warnings

In [ ]:
market_input = input("Nhập thị trường tương ứng (United States or Canada): ")
ad_type = input("Nhập ad_type tương ứng (SP, SD, SB): ")
file_name_output = input("Nhập tên file Excel muốn lưu: ")
uploaded = files.upload()

Nhập thị trường tương ứng (United States or Canada): Canada
Nhập ad_type tương ứng (SP, SD, SB): SD
Nhập tên file Excel muốn lưu: a


Saving Sponsored Display Campaign report.xlsx to Sponsored Display Campaign report (1).xlsx


In [ ]:
dfs = {}

for file_name in uploaded.keys():
    if file_name.endswith('.csv'):
        dfs[file_name] = pd.read_csv(file_name)
    elif file_name.endswith('.xlsx') or file_name.endswith('.xls'):
        xls = pd.ExcelFile(file_name, engine='openpyxl')
        default_sheet_name = xls.sheet_names[0]
        dfs[file_name] = pd.read_excel(file_name, sheet_name=default_sheet_name, engine='openpyxl')

first_file_name = next(iter(dfs))
report = dfs[first_file_name]

/usr/local/lib/python3.10/dist-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [ ]:
product_id = pd.read_html('https://docs.google.com/spreadsheets/d/e/2PACX-1vQKItZA2bNZCY2yD52UEunCjkuq8e9yDuHLQzLqbOvLy13GeJWLWFmujTCRbrBVNA/pubhtml?gid=678803432&single=true', skiprows=1)[0]
product_id = product_id.iloc[1:, 1:]

In [ ]:
  product_id['SKU'] = product_id['SKU'].astype(str)
  def get_matching_sku(campaign_name):
      for sku in product_id['SKU']:
          if sku in campaign_name:
              return sku
      return None

  report['SKU'] = report['Campaign Name'].apply(get_matching_sku)

In [ ]:
trim_column_names = lambda x: x.strip()
report.rename(columns=trim_column_names, inplace=True)

if ad_type == 'SP':
    report.rename(columns={'Click-Thru Rate (CTR)':'CTR',
                     'Cost Per Click (CPC)':'CPC',
                     '7 Day Total Orders (#)':'Orders',
                     'Total Advertising Cost of Sales (ACOS)':'ACOS',
                     'Total Return on Advertising Spend (ROAS)':'ROAS',
                     '7 Day Total Sales':'Sales'}, inplace=True)

elif ad_type == 'SB':
    report.rename(columns={'Click-Thru Rate (CTR)':'CTR',
                         'Cost Per Click (CPC)':'CPC',
                         '14 Day Total Orders (#)':'Orders',
                         'Total Advertising Cost of Sales (ACOS)':'ACOS',
                         'Total Return on Advertising Spend (ROAS)':'ROAS',
                         '14 Day Total Sales':'Sales'}, inplace=True)

elif ad_type =='SD':
    report.rename(columns={'Click-Thru Rate (CTR)':'CTR',
                         'Cost Per Click (CPC)':'CPC',
                         '14 Day Total Orders (#)':'Orders',
                         'Total Advertising Cost of Sales (ACOS)':'ACOS',
                         'Total Return on Advertising Spend (ROAS)':'ROAS',
                         '14 Day Total Sales':'Sales'}, inplace=True)

In [ ]:
  if ad_type != "SB" and report['Budget'].dtype == 'object':

    report['Budget'] = report['Budget'].astype(str)

    report['Budget'] = report['Budget'].str.replace("CA", "").str.replace("US", "").str.replace("$", "").str.replace(",", "")

    report['Budget'] = report['Budget'].astype(float)

if report['Spend'].dtype == 'object':

    report['Spend'] = report['Spend'].astype(str)

    report['Spend'] = report['Spend'].str.replace("CA", "").str.replace("US", "").str.replace("$", "").str.replace(",", "")

    report['Spend'] = report['Spend'].astype(float)

if report['CPC'].dtype == 'object':

    report['CPC'] = report['CPC'].astype(str)

    report['CPC'] = report['CPC'].str.replace("CA", "").str.replace("US", "").str.replace("$", "").str.replace(",", "")

    report['CPC'] = report['CPC'].astype(float)

if report['Sales'].dtype == 'object':

    report['Sales'] = report['Sales'].astype(str)

    report['Sales'] = report['Sales'].str.replace("CA", "").str.replace("US", "").str.replace("$", "").str.replace(",", "")

    report['Sales'] = report['Sales'].astype(float)

In [ ]:
report['Product Number'] = np.nan
report['SKU'] = report['SKU'].astype(str)

In [ ]:
report['Cost Type'] = report['Campaign Name'].apply(lambda x: 'CPM' if 'CPM' in x else 'CPC')

if ad_type == "SP":
  def classify_campaign_name(campaign_name):
    words = campaign_name.split()

    if "Auto" in words:
        return "Auto"
    elif "Query" in words or "query" in words:
        return "SP Query"
    elif "jido" in words:
        return "Auto"
    elif "Research" in words:
        return "Research"
    elif "Performance" in words:
        return "Performance"
    elif "term" in words or "terms" in words:
        return "search terms"
    elif "TD" in words:
        return "TD"
    elif "Broad" in words:
        return "SP Broad"
    elif "Exact" in words or "EX8" in words:
        return "SP Exact"
    elif "TOS" in words:
        return "TOS"
    elif "PP" in words:
        return "PP"
    elif "PT" in words:
        return "SP PT"
    else:
        return "SP Phrase"

elif ad_type == "SB":
  def classify_campaign_name(campaign_name):
    if "Video Ads Phrase" in campaign_name:
        return "SB Video Phrase"
    elif "Video Ads Broad" in campaign_name:
        return "SB Video Broad"
    elif "SB Video Exact" in campaign_name:
        return "SB Video Exact"
    elif "Video Ads PT" in campaign_name:
        return "SB Video PT"
    elif "Video Ads Query" in campaign_name:
        return "SB Video Query"
    else:
        return "SB"

if ad_type == "SD":
  report['Campaign Form'] = 'SD PT'
else:
  report['Campaign Form'] = report['Campaign Name'].apply(classify_campaign_name)

if market_input == "Canada":
  report['Spend'] = report['Spend'] * 0.76
  report['Sales'] = report['Sales'] * 0.76

report['Market'] = market_input
report['Brand'] = "BlueStars"

if ad_type == "SB":
  report["Campaign Type"] = 'Sponsor Brands'
  report["Status"] = np.nan
  report["Budget"] = np.nan
  report["Targeting Type"] = np.nan
  report["Bidding strategy"] = np.nan

elif ad_type == "SD":
  report["Campaign Type"] = 'Sponsor Display'
  report["Targeting Type"] = np.nan
  report["Bidding strategy"] = np.nan

report['Start Date'] = np.nan
report['End Date'] = np.nan

order1 = ['Date', 'Campaign Type', 'Campaign Name','Bidding strategy',
         'Impressions','Clicks', 'Spend',
         'Orders', 'Sales', 'Product Number',
         'Campaign Form', 'Market', 'Brand', 'Cost Type', 'SKU']

data_merge1 = report[order1]

In [ ]:
  if 'SKU' in report.columns:
      sku_none = (report['SKU'] == 'None').any()

      if sku_none:
          print("WARNING: Cột 'SKU' có giá trị None, cần kiểm tra lại!!!")
      else:
          print("Dữ liệu hoàn chỉnh. Không cần kiểm tra")
  else:
      print("Không tìm thấy cột 'SKU' trong báo cáo.")

Dữ liệu hoàn chỉnh. Không cần kiểm tra


In [ ]:
with pd.ExcelWriter(file_name_output + ".xlsx") as writer:
    data_merge1.to_excel(writer, sheet_name='Sheet1', index=False)

files.download(file_name_output + ".xlsx")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>